In [3]:
!pip install underthesea

In [4]:
import pickle
import random
import string
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from underthesea import word_tokenize

In [5]:
path = '/content/drive/MyDrive/NLP Training/'

In [6]:
corpus = open(path + 'datasets/real_estates.txt', 'r', encoding='utf-8').read().lower()
big_corpus = open(path + 'datasets/big_corpus.txt', 'r', encoding='utf-8').read().lower()
corpus += big_corpus[:1000000]
len(corpus)

1551269

In [7]:
def clean_doc(doc):
  replace_with_2_spaces = open(path + 'datasets/replace_with_2_spaces.txt', 'r', encoding='utf-8').read().split('\n')
  for val in replace_with_2_spaces:
    old, new = val.split(' ', 1)
    doc = doc.replace(old, new) 
  table = str.maketrans('', '', string.punctuation)
  doc = doc.translate(table)
  tokens = [word for word in word_tokenize(doc) if word.isalpha()]
  return tokens

In [8]:
tokens = clean_doc(corpus)
len(tokens)

176071

In [9]:
print('Len tokens: %d' % len(tokens))
print('Unique tokens: %d' % len(set(tokens)))

Len tokens: 176071
Unique tokens: 4930


In [10]:
max_len = 51
step = 1
# create a list of line
sequences = [' '.join(tokens[i:i + max_len]) for i in range(0, len(tokens) - max_len, step)]
with open(path + 'datasets/sequences.txt', 'w', encoding='utf-8') as f:
  f.write('\n'.join(sequences))

In [11]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)
seq_vectors = tokenizer.texts_to_sequences(sequences)

In [12]:
vocab_size = len(tokenizer.word_index) + 1
len(tokenizer.word_index)

4930

In [13]:
seq_vectors = np.array(seq_vectors)
X, y = seq_vectors[:,:-1], seq_vectors[:,-1]
# one hot encoder 
y = to_categorical(y, num_classes=vocab_size)

In [14]:
seq_length = X.shape[1]
len_line = max_len - 1 
batch_size = 64

In [15]:
model = Sequential()
model.add(Embedding(vocab_size, len_line, input_length=seq_length))
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(256))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            246550    
_________________________________________________________________
lstm (LSTM)                  (None, 50, 256)           314368    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense (Dense)                (None, 100)               25700     
_________________________________________________________________
dense_1 (Dense)              (None, 4931)              498031    
Total params: 1,609,961
Trainable params: 1,609,961
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, batch_size=batch_size, epochs=100)

Epoch 1/100
2751/2751 [==============================] - 49s 15ms/step - loss: 6.5002 - accuracy: 0.0211
Epoch 2/100
2751/2751 [==============================] - 41s 15ms/step - loss: 5.9279 - accuracy: 0.0466
Epoch 3/100
2751/2751 [==============================] - 40s 15ms/step - loss: 5.5575 - accuracy: 0.0871
Epoch 4/100
2751/2751 [==============================] - 40s 15ms/step - loss: 5.2533 - accuracy: 0.1293
Epoch 5/100
2751/2751 [==============================] - 41s 15ms/step - loss: 5.0131 - accuracy: 0.1653
Epoch 6/100
2751/2751 [==============================] - 41s 15ms/step - loss: 4.8254 - accuracy: 0.1898
Epoch 7/100
2751/2751 [==============================] - 41s 15ms/step - loss: 4.6757 - accuracy: 0.2075
Epoch 8/100
2751/2751 [==============================] - 41s 15ms/step - loss: 4.5517 - accuracy: 0.2212
Epoch 9/100
2751/2751 [==============================] - 40s 15ms/step - loss: 4.4463 - accuracy: 0.2332
Epoch 10/100
2751/2751 [==============================]

In [ ]:
model.save(path + 'models/lstm_word_base.h5')
pickle.dump(tokenizer, open(path + 'models/word_base_tokenizer.pkl', 'wb'))

In [ ]:
seed_text = 'mua bán nhà đất giá rẻ tại thành phố hồ chí minh'
print(seed_text)

In [ ]:
encoded = tokenizer.texts_to_sequences([seed_text])[0] # encode seed_text
yhat = model.predict(encoded, verbose=0) # predict

In [ ]:
yhat = np.argmax(yhat)

In [ ]:
# reverse the word_index
word_index = {val: key for key, val in tokenizer.word_index.items()}

In [ ]:
yhat

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
def generate_text(text, max_len=100):
  for _ in range(max_len):
    encoded = tokenizer.texts_to_sequences([text])[0]
    encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
    yhat = model.predict(encoded, verbose=0)
    yhat = np.argmax(yhat)
    if yhat in word_index.keys():
      text += ' ' + word_index[yhat]
  return text  

In [ ]:
generate_text('bất động sản thành phố', 100)